In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import warnings
import faiss
import os

c:\Users\ASUS\AppData\Local\Programs\Python\Python38\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
# CONSTANT

DATA_DIR = "data/"
DATA_PATH = os.path.join(DATA_DIR, "data.csv")
INDEX_PATH = os.path.join(DATA_DIR, "karya.index")
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

def warn(*args, **kwargs):
    pass
warnings.warn = warn

In [3]:
data = pd.read_csv(DATA_PATH)

In [4]:
data

,uuid,key
0,Cinta yang Tak Pernah Pudar,Cinta yang Tak Pernah Pudar oleh Anita Dewi. K...
1,Petualangan di Bumi Tengah,Petualangan di Bumi Tengah oleh Joko Susilo. S...
2,Rahasia Alam Semesta,Rahasia Alam Semesta oleh Siti Rahayu. Menguak...
3,Dunia Tanpa Akhir,Dunia Tanpa Akhir oleh Budi Santoso. Dalam dun...
4,Melodi Senja,Melodi Senja oleh Dian Pratiwi. Sebuah kumpula...
5,Perangkap Waktu,Perangkap Waktu oleh Ahmad Surya. Ditawan dala...
6,Mimpi di Tepi Laut,Mimpi di Tepi Laut oleh Rani Wijaya. Cerita te...
7,Cahaya Dalam Kegelapan,Cahaya Dalam Kegelapan oleh Ani Susanti. Sebua...
8,Pergulatan Jiwa,Pergulatan Jiwa oleh Eka Putra. Kisah tentang ...
9,Rahasia Sang Pencipta,Rahasia Sang Pencipta oleh Fitri Indah. Mengun...


In [5]:
model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')

c:\Users\ASUS\AppData\Local\Programs\Python\Python38\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
encoded_data = model.encode(data.key.tolist())
encoded_data = np.asarray(encoded_data.astype('float32'))
index = faiss.IndexIDMap(faiss.IndexFlatIP(512))
index.add_with_ids(encoded_data, np.array(range(0, len(data))))
faiss.write_index(index, INDEX_PATH)

In [7]:
def fetch_karya(dataframe_idx):
    info = data.iloc[dataframe_idx]
    meta = dict()
    meta['uuid'] = info['uuid']
    return meta
    
def search(query, top_k, index, model):
    query_vector = model.encode([query])
    top_k = index.search(query_vector, top_k)
    result_id = top_k[1].tolist()[0]
    result_id = list(np.unique(result_id))
    results =  [fetch_karya(idx) for idx in result_id]
    return results

In [8]:
query="Liar"
results=search(query, top_k=2, index=index, model=model)
print("\n")
for result in results:
    print('\t',result)



	 {'uuid': 'Dunia dalam Genggaman'}
	 {'uuid': 'Langit di Mata'}


In [9]:
def add_karya(uuid, text):
    global data
    encoded_data = model.encode([text])
    encoded_data = np.asarray(encoded_data.astype('float32'))
    index.add_with_ids(encoded_data, np.array(range(len(data), len(data) + 1)))
    faiss.write_index(index, INDEX_PATH)

    new_record = pd.DataFrame({'uuid': [uuid], 'key': [text]})

    # Concatenate the new record to the DataFrame
    data = pd.concat([data, new_record], ignore_index=True)
    data.to_csv(DATA_PATH, index=False)

In [10]:
add_karya("Petualangan di Alam Liar","Petualangan di Alam Liar oleh Bambang Wijaya. Menjelajahi keindahan dan tantangan alam liar, menghadapi hewan-hewan buas dan mengungkap misteri di balik kehidupan liar. Genre: Petualangan, Alam Liar")
# "Petualangan di Alam Liar","Petualangan di Alam Liar oleh Bambang Wijaya. Menjelajahi keindahan dan tantangan alam liar, menghadapi hewan-hewan buas dan mengungkap misteri di balik kehidupan liar. Genre: Petualangan, Alam Liar"

In [11]:
query="Liar"
results=search(query, top_k=2, index=index, model=model)
print("\n")
for result in results:
    print('\t',result)



	 {'uuid': 'Dunia dalam Genggaman'}
	 {'uuid': 'Petualangan di Alam Liar'}
